# Segundo punto

Juan Sebastian Espitia

Anamaria Irmgard Mojica 201316580

María José Ocampo 201719100

Leidy Romero Santana 201714832

### Enunciado:
Comparación de estrategias de motores de búsqueda

A continuación, implementará un motor de búsqueda con cuatro estrategias diferentes.
1. Búsqueda binaria (BS).
2. Búsqueda binaria con índice invertido (BSII)
3. Recuperación clasificada básica (RRI)
4. Recuperación clasificada y vectorización de documentos (RRDV)
Debe hacer su propia implementación utilizando numpy y pandas para manejar matrices y matrices.

Nota: Hay puntos adicionales [15p] si su implementación de "índice invertido" está distribuida (usando para
ejemplo MapReduce) o la clasificación de discos eficiente se realiza mediante BSBI. Ambas estrategias se explican en
capítulo 4 del libro https://nlp.stanford.edu/IR-book/pdf/04const.pdf.
Conjunto de datos: hay tres archivos que componen el conjunto de datos. "Docs raws texts" contiene 331 documentos en
formato NAF (XML: debe utilizar el título y el contenido para modelar cada documento). "Queries raw text" contiene
35 consultas. "relevance-judgments.tsv" contiene para cada consulta los documentos relevantes. Estos documentos relevantes fueron construidos manualmente por jueces humanos y sirven como conjunto de datos de evaluación.

Pasos de preprocesamiento: para los siguientes puntos, debe preprocesar documentos y consultas utilizando tokenización por palabra, eliminación de stop words, normalización y stemming. 

Primero se instalan los paquetes necesarios

In [138]:
!pip install gensim

A. Implemente el modelo BOW usando gensim y el conjunto de datos de 331 documentos. Guardar el diccionario y el corpus en formato Matrix Market.

B. Cargue el corpus y transfórmelo en un modelo Tf idf  (es decir, documentos como vectores tf.idf). Serializar el modelo resultante a disco.

C. Prepare el modelo para realizar algunas consultas de similitud. Cree un ¿¿¿índice Matriz de similitud??? y guárdelo en el disco.

A. Para cada una de las 35 consultas en el conjunto de datos, recupere los documentos clasificados (ordenados por el puntaje de similitud del coseno) (incluya solo los documentos con una puntuación superior a 0 para una consulta determinada). Escribe un archivo (GENSIM-queries_results) con los resultados siguiendo el mismo formato que el archivo "relevance-judgments":

B. Evaluación de resultados. Calcule P @ M, R @ M, NDCG @ M por consulta. M es el número de documentos relevants encontrados en el archivo relevance-judgments por consulta. Luego calcule MAP como métrica general.

NOTA I: Para P @ M y R @ M, suponga una escala de relevancia binaria. Documentos no encontrados en relevancia relevance-judgments file NO son relevantes para una consulta determinada.

NOTA II: Para NDCG @ M use la escala de relevancia no binaria que se encuentra en el archivo de juicios de relevancia. 

In [139]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open
import nltk
nltk.download('punkt')


import csv
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO
from bs4 import BeautifulSoup
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [140]:
text_corpus_en = []
with open('/content/normalized_docs.txt', 'r') as f:
    for line in f:
      if line != '\n':
        text_corpus_en.append(line.replace(u'\xa0', u' '))
text_corpus_en

['william beaumont and the human digestion.  william beaumont: physiology of digestion image source.  on november 21, 1785, us american surgeon william beaumont was born. he became best known as “father of gastric physiology” following his research on human digestion. william beaumont was born in lebanon, connecticut and became a physician. he served as a surgeon’s mate in the army during the war of 1812. he opened a private practice in plattsburgh, new york, but rejoined the army as a surgeon in 1819. beaumont was stationed at fort mackinac on mackinac island in michigan in the early 1820s when it existed to protect the interests of the american fur company. the fort became the refuge for a wounded 19-year old french canadian fur trader named alexis st. martin when a shotgun went off by accident in the american fur company store at close range june 6th, 1822. st. martin’s wound was quite serious because his stomach was perforated and several ribs were broken. nobody really expected th

In [141]:
p = PorterStemmer()

def process(text):
    doc_nor = text.lower()
    doc_sw = remove_stopwords(doc_nor)
    doc_stem = p.stem_sentence(doc_sw) 
    return nltk.word_tokenize(doc_stem)

docDict = []
for doc in text_corpus_en:
    docDict.append(process(doc))

docDict[0]

['william',
 'beaumont',
 'human',
 'digestion',
 '.',
 'william',
 'beaumont',
 ':',
 'physiolog',
 'digest',
 'imag',
 'source',
 '.',
 'novemb',
 '21',
 ',',
 '1785',
 ',',
 'american',
 'surgeon',
 'william',
 'beaumont',
 'born',
 '.',
 'best',
 'known',
 '“',
 'father',
 'gastric',
 'physiology',
 '”',
 'follow',
 'research',
 'human',
 'digestion',
 '.',
 'william',
 'beaumont',
 'born',
 'lebanon',
 ',',
 'connecticut',
 'physician',
 '.',
 'serv',
 'surgeon',
 '’',
 'mate',
 'armi',
 'war',
 '1812.',
 'open',
 'privat',
 'practic',
 'plattsburgh',
 ',',
 'new',
 'york',
 ',',
 'rejoin',
 'armi',
 'surgeon',
 '1819.',
 'beaumont',
 'station',
 'fort',
 'mackinac',
 'mackinac',
 'island',
 'michigan',
 'earli',
 '1820',
 'exist',
 'protect',
 'interest',
 'american',
 'fur',
 'company',
 '.',
 'fort',
 'refug',
 'wound',
 '19-year',
 'old',
 'french',
 'canadian',
 'fur',
 'trader',
 'name',
 'alexi',
 'st.',
 'martin',
 'shotgun',
 'went',
 'accid',
 'american',
 'fur',
 'compa

In [142]:
## Dictionary == Vocab

dictionary = corpora.Dictionary(docDict)
dictionary.save('midict.dict')
print(dictionary)

print(dictionary.token2id)

Dictionary(17728 unique tokens: [',', '.', '1785', '1812.', '1819.']...)
{',': 0, '.': 1, '1785': 2, '1812.': 3, '1819.': 4, '1820': 5, '1822.': 6, '1833.': 7, '19-year': 8, '21': 9, '6th': 10, '83.': 11, ':': 12, 'accid': 13, 'acid': 14, 'activ': 15, 'affect': 16, 'ag': 17, 'alexi': 18, 'also': 19, 'american': 20, 'armi': 21, 'basi': 22, 'beaumont': 23, 'best': 24, 'better': 25, 'bit': 26, 'book': 27, 'born': 28, 'break': 29, 'broken': 30, 'canadian': 31, 'caus': 32, 'chemic': 33, 'chemical': 34, 'children': 35, 'close': 36, 'compani': 37, 'company': 38, 'complet': 39, 'connecticut': 40, 'consid': 41, 'di': 42, 'did': 43, 'differ': 44, 'digest': 45, 'digestion': 46, 'discov': 47, 'earli': 48, 'emotions': 49, 'examin': 50, 'exist': 51, 'expect': 52, 'experi': 53, 'famou': 54, 'father': 55, 'find': 56, 'fistula': 57, 'follow': 58, 'food': 59, 'fort': 60, 'french': 61, 'fur': 62, 'fuse': 63, 'gain': 64, 'gastric': 65, 'gave': 66, 'healed': 67, 'hole': 68, 'human': 69, 'hydrochlor': 70, '

In [143]:
##  Market Matrix format
# Step 1: Build the corpus from big file
class MyCorpus(object):
    def __iter__(self):
        for line in smart_open("/content/normalized_docs.txt","rb"):
          if line != '\n':
            #print(line.decode("utf-8"))
            yield dictionary.doc2bow(process(line))

corpus_memory_friendly = MyCorpus()
corpora.MmCorpus.serialize("corpus.mm",corpus_memory_friendly)

In [144]:
corpus = corpora.MmCorpus("corpus.mm")

In [145]:
# Step 2: Build Tfidf model from corpus
dictionary = corpora.Dictionary.load('midict.dict')
corpus = corpora.MmCorpus('corpus.mm')
tfidf = models.TfidfModel(corpus)

# TODO: Serialize tf idf model to disk










In [146]:
# Step 3: Similarity Matrix
# In this step the index is created
index = similarities.MatrixSimilarity(tfidf[corpus])
index.save('similmatrix.index')

In [147]:
request_url_2 = urlopen('https://github.com/LeidyRomero/data_nlp/raw/main/HW01/queries-raw-texts.zip') 
z2 = ZipFile(BytesIO(request_url_2.read()))

index = similarities.MatrixSimilarity.load('similmatrix.index')
ret = {}

# Se escribe el archivo GENSIM-queries_results
with open('/content/GENSIM-queries_results.tsv', 'wt') as out_file:
  tsv_writer = csv.writer(out_file, delimiter='\t')
      
  for name in z2.namelist():
    soup = BeautifulSoup(z2.read(name),'xml')
    print(soup.public.attrs.get('publicId')+': '+soup.raw.string)
    id_query = soup.public.attrs.get('publicId').split(':')[0]
    # You can transform any doc using your model
    #query = soup.raw.string
    query_doc_bow = dictionary.doc2bow(process(soup.raw.string)) # Important: Same corpus preprocess
    #print(query_doc_bow)
    print(tfidf[query_doc_bow]) # Model transformation

    sims = index[tfidf[query_doc_bow]]
    result = sorted(list(enumerate(sims,1)), key=lambda x: x[1], reverse = True)
    ret[id_query] = result
    resultTopTen = result[:10]
    print(resultTopTen)
    print('-'*200)
    
    w = ''
    for pair in resultTopTen:
      if pair[1]>0:
        w += str(pair[0])+':'+str(pair[1])+','
        
    tsv_writer.writerow([id_query, w[:-1]])

q01: Fabrication of music instruments
[(698, 0.4785475819504571), (1234, 0.7316664619034943), (2295, 0.48544453888677447)]
[(16, 0.10480724), (259, 0.07017432), (254, 0.057479233), (186, 0.057007812), (85, 0.049024507), (209, 0.046648655), (170, 0.038198512), (153, 0.036386117), (8, 0.033125326), (215, 0.029164989)]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
q02: famous German poetry
[(54, 0.22183214163992113), (256, 0.9249770617605123), (820, 0.3085578327515529)]
[(147, 0.101807825), (283, 0.06377807), (149, 0.04761839), (134, 0.04123834), (2, 0.039731566), (23, 0.03135969), (293, 0.027454864), (14, 0.026566606), (52, 0.024561642), (282, 0.024512501)]
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [148]:
# . Calcule P@M (relevancia binaria), R@M (relevancia binaria), NDCG@M y MAP por consulta,
# M numero de documentos relevantes.

# Lectura de relevance-judgments
request_url_3 = urlopen('https://github.com/LeidyRomero/data_nlp/raw/main/HW01/relevance-judgments.tsv') 
z3 = request_url_3.read()

f = open("/content/relevance-judgments.tsv", "wb")
f.write(z3)
f.close()

tsv = open("/content/relevance-judgments.tsv","r")
read_tsv = csv.reader(tsv, delimiter="\t")

sum_avg_precision = 0
Q = 46

for row in read_tsv:
  #query
  print('-'*50)
  print('-'*23+row[0]+'-'*23)
  print('-'*50)
  #docs rel
  #print(row[1])
  M = len(row[1].split(","))
  #docs rel
  rel = {}
  for r in row[1].split(","):
    id = r.split(":")[0]
    if id[1]!='0':
      id = int(id[-3:])
    elif id[1]=='0' and id[2]!='0':
      id = int(id[-2:])
    else:
      id = int(id[-1:])
    rel[id] = r.split(":")[1]
  rel_docs = list(rel.keys())

  DCG_ACUM = 0
  BEST_DCG_ACUM = 0
  #BEST DCG
  BEST_DCG = list(rel.values())
  BEST_DCG.sort(reverse = True)

  sum_precision = 0
  cont_precision = 0
  numerador = None
  antiguo_numerador = None

  for i in range(0,M):
    if numerador is not None:
      antiguo_numerador = numerador

    ret_ids = [pair[0] for pair in ret[row[0]][:i+1]]
    numerador = [value for value in rel_docs[:i+1] if value in ret_ids] 

    if antiguo_numerador is not None and len(antiguo_numerador)!=len(numerador):
      sum_precision += len(numerador)/(i+1)
      print('entra')

    print('antiguo: '+str(antiguo_numerador))
    print('nuevo: '+str(numerador))

    print('Rel = '+str(rel_docs[:i+1])+', Ret = '+str(ret[row[0]][:i+1]))
    print('P@'+str(i+1)+': '+str(len(numerador)/(i+1)))
    #print('R@'+str(i+1)+': '+str(len(numerador)/M))
      
    discount_factor = 1/(np.log2(np.max(np.array([i+1,2]))))
    #DCG 
    rel_i = rel.get(ret_ids[i])
    if rel_i is not None:
      DCG_ACUM += discount_factor*int(rel_i)

    #BEST_DCG_2
    BEST_DCG_ACUM += discount_factor*int(BEST_DCG[i])

    #print('NDCG@'+str(i+1)+': '+str(DCG_ACUM/BEST_DCG_ACUM))

    print('-'*20)
  sum_avg_precision += sum_precision/M
print('MAP: '+ str(sum_avg_precision/Q))
tsv.close()

--------------------------------------------------
-----------------------q01-----------------------
--------------------------------------------------
antiguo: None
nuevo: []
Rel = [186], Ret = [(16, 0.10480724)]
P@1: 0.0
--------------------
antiguo: []
nuevo: []
Rel = [186, 254], Ret = [(16, 0.10480724), (259, 0.07017432)]
P@2: 0.0
--------------------
entra
antiguo: []
nuevo: [254, 16]
Rel = [186, 254, 16], Ret = [(16, 0.10480724), (259, 0.07017432), (254, 0.057479233)]
P@3: 0.6666666666666666
--------------------
--------------------------------------------------
-----------------------q02-----------------------
--------------------------------------------------
antiguo: None
nuevo: []
Rel = [136], Ret = [(147, 0.101807825)]
P@1: 0.0
--------------------
antiguo: []
nuevo: []
Rel = [136, 139], Ret = [(147, 0.101807825), (283, 0.06377807)]
P@2: 0.0
--------------------
antiguo: []
nuevo: []
Rel = [136, 139, 143], Ret = [(147, 0.101807825), (283, 0.06377807), (149, 0.04761839)]
P@3: